# CIFAR-10 (RNN)

In [2]:
import os
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Input, RNN, SimpleRNN, Embedding
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.activations import relu, softmax, tanh
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed

In [3]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

Version de TensorFlow : 2.6.0
Nom du GPU :Metal device set to: Apple M1
 /device:GPU:0


2022-01-02 00:01:09.371788: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-02 00:01:09.371871: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Importation du dataset

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [6]:
x_train = to_categorical(x_train.astype('float32') / 256)
x_test = to_categorical(x_test.astype('float32') / 256)

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [7]:
LOG_DIR = os.path.join("logs")
MODEL_DIR = os.path.join("models")

## Fixer les seeds

In [8]:
set_seed(42) # TensorFlow
seed(42) # NumPy

---

# RNN

![RNN](https://ichi.pro/assets/images/max/724/1*go8PHsPNbbV6qRiwpUQ5BQ.png)

Voici à quoi ressemble un RNN (Reccurent Neural Network). La particularité du RNN est qu'il s'appelle lui-même

Pour voir l'influence des hyperparamètres sur un RNN, nous avons d'abord besoin de fixer le nombre d'époques et le nombre de layer sur notre couche de sortie.

In [9]:
EPOCHS = 1000
SHUFFLE = True
NUM_CLASSES = 10

Ce modèle n'a pas était vu en cours, du coup, nous allons nous faciliter la tâche, nous allons créer un modèle principal appelé main puis nous allons tester un ou plusieurs hyperparamètres à la fois pour que l'on puisse aperçevoir leur(s) influence sur le modèle.

## Modèle main

Le modèle main est notre modèle principale, ce sera celui avec lequelle nous comparerons tout les autres prochains modèles et essayer de distinguer le plus clairement possible l'influence des hyperparamètres que nous modifierons. Nous avons choisi les hyperparamètres par défaut lorsque c'était possible et avons choisi le reste selon notre intuition.

In [10]:
def RNN_main(num_units):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layer = Flatten()(input_layer)
    hidden_layer = Embedding(32, 32)(hidden_layer)
    hidden_layer = SimpleRNN(num_units)(hidden_layer)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layer)
    return Model(input_layer, output_layer)

Voici les hyperparamètres de notre modèle :

In [11]:
batch_size = 32
num_units = 4
learning_rate = 0.01
momentum = 0

Tout au long de notre section sur le RNN, nous n'utiliserons comme optimiseur uniquement Stochastic Gradient Descent (SGD). Nous pouvons désormais instancier et entraîner notre modèle.

In [ ]:
rnn = RNN_main(num_units)
print(rnn.summary())

rnn.compile(
    loss=categorical_crossentropy,
    optimizer=SGD(learning_rate=learning_rate, momentum=momentum),
    metrics=categorical_accuracy
           )

RNN_LOG = os.path.join(LOG_DIR, "rnn",
                f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mom_{momentum}_af_tanh_units_{num_units}_main")

RNN_MODEL = os.path.join(MODEL_DIR, "rnn",
                f"ep_{EPOCHS}_bs_{batch_size}_opt_SGD_lr_{learning_rate}_mom_{momentum}_af_tanh_units_{num_units}_main.keras")

rnn.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=EPOCHS,
        validation_data=(x_test, y_test),
        shuffle=SHUFFLE,
        callbacks=[TensorBoard(RNN_LOG)]
       )
rnn.save(RNN_MODEL)


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 3072, 32)          1024      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 4)                 148       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                50        
Total params: 1,222
Trainable params: 1,222
Non-trainable params: 0
_________________________________________________________________
None


2022-01-02 00:02:18.874926: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-02 00:02:18.874937: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-01-02 00:02:18.875087: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-01-02 00:02:19.138742: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-02 00:02:19.138947: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-02 00:02:19.315940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/1000
